In [3]:
cd drive/MyDrive/hackerearth/

/content/drive/MyDrive/hackerearth


In [6]:
ls

dbert_inp.pkl    dbert_model/           train.csv
dbert_label.pkl  sample_submission.csv  uncased_L-12_H-768_A-12/
dbert_mask.pkl   test.csv               uncased_L-12_H-768_A-12.zip


In [7]:
!pip install spacy

In [8]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9 MB 47.0 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [1]:
import pandas as pd
import csv
import nltk
import unicodedata
import re
from nltk.corpus import stopwords

In [2]:
import numpy as np
import spacy


# Need to load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

In [4]:
import sklearn

In [5]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
df = pd.read_csv("train.csv")

In [25]:
tdf = pd.read_csv('test.csv')

In [8]:
df.head()

,video_id,category_id,description
0,7NcVNt7zQJM,10.0,Music video by Luke Bryan performing Out Of No...
1,hNsd-m30m2E,24.0,thanks for watching!!! happy holidays everyone...
2,yLdav-8af6g,1.0,Marcus and Adam build their own custom lightsa...
3,Ii3Rr4-Pxow,26.0,Make sure you subscribe to my channel and hit ...
4,515lLZS082U,24.0,It's a riff-off pitches! Get Tix: http://unvrs...


In [9]:
df['description']=df['description'].map(preprocess_sentence)

In [27]:
tdf['description']=tdf['description'].map(preprocess_sentence)

In [10]:
df.shape

(3471, 3)

In [11]:
df.head()

,video_id,category_id,description
0,7NcVNt7zQJM,10.0,music video luke bryan performing nowhere girl...
1,hNsd-m30m2E,24.0,thanks watching happy holidays everyone links ...
2,yLdav-8af6g,1.0,marcus adam build custom lightsabers always tw...
3,Ii3Rr4-Pxow,26.0,make sure subscribe channel hit notification b...
4,515lLZS082U,24.0,riff pitches get tix http unvrs tix pitchperfe...


In [12]:
df = df.dropna()

In [13]:
df.shape

(3471, 3)

In [14]:
with nlp.disable_pipes():
  doc_vectors = np.array([nlp(text).vector for text in df.description])

In [30]:
with nlp.disable_pipes():
  tdoc_vectors = np.array([nlp(text).vector for text in tdf.description])

In [15]:
doc_vectors.shape

(3471, 300)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, df.category_id	, test_size=0.1, random_state=1)

In [ ]:
from sklearn.svm import LinearSVC

# Set dual=False to speed up training, and it's not needed
svc = LinearSVC(random_state=1, dual=False, max_iter=1000)
svc.fit(X_train, y_train)
print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%", )

Accuracy: 67.241%


In [ ]:
 y_pred=svc.predict(X_test)

In [ ]:
sklearn.metrics.f1_score(y_test, y_pred,average='micro')

0.6724137931034483

In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier()

In [ ]:
clf.fit(X_train, y_train)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [ ]:
clf.score(X_test, y_test)

0.7701149425287356

In [ ]:
 y_pred=clf.predict(X_test)

In [ ]:
sklearn.metrics.f1_score(y_test, y_pred,average='micro')

0.7701149425287356

In [18]:
!pip install XGboost

In [19]:
from xgboost import XGBClassifier

In [20]:
my_model = XGBClassifier()
my_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [31]:
tdf_pred = my_model.predict(tdoc_vectors)

In [21]:
my_model.score(X_test, y_test)

0.7816091954022989

In [22]:
ls

dbert_inp.pkl    dbert_model/           train.csv
dbert_label.pkl  sample_submission.csv  uncased_L-12_H-768_A-12/
dbert_mask.pkl   test.csv               uncased_L-12_H-768_A-12.zip


In [23]:
ls

dbert_inp.pkl    dbert_model/           train.csv
dbert_label.pkl  sample_submission.csv  uncased_L-12_H-768_A-12/
dbert_mask.pkl   test.csv               uncased_L-12_H-768_A-12.zip


In [26]:
tdf.head()

,video_id,description
0,2kyS6SvSYSE,SHANTELL'S CHANNEL - https://www.youtube.com/s...
1,_6lGaYh71g4,Balmuda: the Toaster is a Japanese invention t...
2,X_tYrnv_o6A,These are the molecular machines inside your b...
3,BEcs3TL8DII,LeBron James is 'obviously' the NBA MVP this s...
4,26qOMpjicGs,Lol - Subscribe to my new channel that will co...


In [32]:
tdf_pred

array([22., 19., 27., ..., 24., 17., 24.])

In [34]:
tdf['category_id'] = pd.Series(tdf_pred, index=tdf.index)

In [35]:
tdf

,video_id,description,category_id
0,2kyS6SvSYSE,shantell channel https www youtube com shantel...,22.0
1,_6lGaYh71g4,balmuda toaster japanese invention brings life...,19.0
2,X_tYrnv_o6A,molecular machines inside body make cell divis...,27.0
3,BEcs3TL8DII,lebron james obviously nba mvp season subscrib...,17.0
4,26qOMpjicGs,lol subscribe new channel continue deliver fun...,24.0
...,...,...,...
1483,2Nlo2ekTRvI,loads better cars cars still good movie whatev...,1.0
1484,tUbky9YzPZY,pond jump fail subscribe http bit subtoroman n...,22.0
1485,I6N-XuYllgA,episode derek stuff friend derek blasberg chal...,24.0
1486,phCIKfJY7SU,raw anniversary deadman emerges wwe first time...,17.0


In [36]:
tdf.to_csv(
    'submission.csv',
    index=False)